In [4]:

import cv2
import numpy as np
import os
from skimage import io

In [5]:

from sklearn.cross_validation import train_test_split
from keras.models import Sequential

C:\Users\HP\Anaconda3\envs\tf\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [6]:
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [7]:
image_path = []
name="path\\to\\folder"
for file_name in os.listdir(name):
    image_path.append(os.path.join(name, file_name))

imageDataFin = []
imageLabels = []
    
# read face images and class labels
for img in image_path:
    imgRead = io.imread(img, as_grey=True)
    imgRead = cv2.resize(imgRead, (250, 250)) 
    imageDataFin.append (imgRead)
    labelRead = int(os.path.split(img)[1].split("_")[0])
#     print(labelRead)
    imageLabels.append(labelRead)

C:\Users\HP\Anaconda3\envs\tf\lib\site-packages\skimage\io\_io.py:49: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')


In [8]:
# split the data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(np.array(imageDataFin), np.array(imageLabels), train_size=0.8, 
                                                    random_state = 123)

# fix the random seed for reproducibility
seed = 123
np.random.seed(seed)

# configuration
batch_size = 32
# nb_classes = 38
nb_classes =35
nb_epoch = 15
# image dimensions
img_rows, img_cols = 250, 250
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

# resdhape the data matrix
if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# scale the data
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# one-hot encoding for the class label
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


X_train shape: (308, 250, 250, 1)
308 train samples
77 test samples


In [ ]:

import keras
import os
from keras.utils import plot_model
os.environ["PATH"] += os.pathsep + 'C:/Users/HP/Anaconda3/envs/tf/Lib/site-packages/graphviz-2.38/release/bin'

# define the convolutional neural network model
model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))


# inputs = Input((250,250))
# model = Model(inputs, model)
print(model.summary())
plot_model(model, to_file = 'C:/Users/HP/Desktop/model/model.png')


# compile the CNN model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the CNN model
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))

# evaluation
score = model.evaluate(X_test, Y_test, verbose=0)
print('Loss:', score[0])
print('Accuracy:', score[1])

# find out which images are classified wrongly
predicted_classes = model.predict_classes(X_test)
correct_classified_indices = np.nonzero(predicted_classes == Y_test)[0]
incorrect_classified_indices = np.nonzero(predicted_classes != Y_test)[0]
if not incorrect_classified_indices:
    print('\nAll test samples are correctly recognized.')
else:
    print('The incorrect indices are:', incorrect_classified_indices) 

C:\Users\HP\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(250, 250,..., padding="valid")`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\HP\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  del sys.path[0]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 248, 248, 32)      320       
_________________________________________________________________
activation_1 (Activation)    (None, 248, 248, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 246, 246, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 246, 246, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 123, 123, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 123, 123, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 484128)            0         
__________

C:\Users\HP\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:37: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 308 samples, validate on 77 samples
Epoch 1/15
308/308 [==============================] - 117s 379ms/step - loss: 8.5734 - acc: 0.0325 - val_loss: 3.5429 - val_acc: 0.0519
Epoch 2/15
308/308 [==============================] - 63s 203ms/step - loss: 3.4758 - acc: 0.0779 - val_loss: 3.4146 - val_acc: 0.0779
Epoch 3/15
308/308 [==============================] - 75s 245ms/step - loss: 2.9857 - acc: 0.2305 - val_loss: 2.8936 - val_acc: 0.3117
Epoch 4/15
308/308 [==============================] - 64s 207ms/step - loss: 2.0701 - acc: 0.4838 - val_loss: 1.9938 - val_acc: 0.5844
Epoch 5/15
308/308 [==============================] - 63s 205ms/step - loss: 1.1109 - acc: 0.7273 - val_loss: 1.4917 - val_acc: 0.6364
Epoch 6/15
308/308 [==============================] - 64s 207ms/step - loss: 0.6063 - acc: 0.8734 - val_loss: 1.1349 - val_acc: 0.7143
Epoch 7/15
288/308 [===========================>..] - ETA: 3s - loss: 0.3522 - acc: 0.9062

In [ ]:
# import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

predicted = model.predict(X_test)
report = classification_report(Y_test, predicted)
print(report)